wav2vec
source: https://huggingface.co/facebook/wav2vec2-base-960h
paper: https://arxiv.org/abs/2006.11477

hasil ujicoba dengan wav2vec2 dirasa kurang memuaskan dikarenakan pada pengujian yang menggunakan rekaman suara yang berbeda tidak menunjukkan setidaknya satu kata yang benar
namun wav2vec2 masih bisa dijelajah lebih lanjut 

alternatif berikutnya akan mencoba menggunakan whisper
source: https://github.com/openai/whisper
paper: https://cdn.openai.com/papers/whisper.pdf

In [10]:

from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import load_dataset
import torch
import io
import speech_recognition as sr
from pydub import AudioSegment
from pydub.playback import play

# load model and tokenizer
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
    
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Found cached dataset librispeech_asr_dummy (/home/nursyah/.cache/huggingface/datasets/patrickvonplaten___librispeech_asr_dummy/clean/2.1.0/f2c70a4d03ab4410954901bde48c54b85ca1b7f9bf7d616e7e2a72b5ee6ddbfc)


In [5]:
def test1():
    path_audio = ds[0]['audio']['path']
    # tokenize
    input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest", sampling_rate=16000).input_values  # Batch size 1

    # retrieve logits
    logits = model(input_values).logits

    # take argmax and decode
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.batch_decode(predicted_ids)

    song = AudioSegment.from_file(path_audio, format='flac')
    print(f"playing sound using  pydub")
    play(song)
    print(transcription)

In [22]:
def test2():
    path_audio = './yt2.wav'

    r = sr.Recognizer()
    with sr.AudioFile(path_audio) as source:
        audio = r.record(source)
        data = io.BytesIO(audio.get_wav_data())
        clip = AudioSegment.from_file(data)
        x = torch.FloatTensor(clip.get_array_of_samples())

        # processing
        inputs = processor(x, return_tensors="pt", padding="longest", sampling_rate=16000).input_values 
        logits = model(inputs).logits

        # take argmax and decode
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = processor.batch_decode(predicted_ids)
        print(transcription)

        
        print(path_audio)
        song = AudioSegment.from_file(path_audio)
        print(f"playing sound using  pydub")
        play(song)

In [21]:
test2()

['HONO LACHAMMM']
./yt2.wav
playing sound using  pydub


ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
